# Scraping modulargrid.net

## Libraries

In [253]:
import json
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import cloudinary
import cloudinary.uploader

## Utility Functions

In [265]:
def wait_for_scroll(old_height, timeout = 10):
    WebDriverWait(driver, timeout).until(
        lambda d: d.execute_script("return document.body.scrollHeight") > old_height
    )

def initialize_driver():
    options = Options()
    options.add_argument("--headless")
    return webdriver.Chrome(options=options)

def chunkify(lst, n):
    return [lst[i::n] for i in range(n)]

## Cloudinary Configuration

In [266]:
cloudinary.config(
    cloud_name = "dglh7onu3",
    api_key = "115324728487844",
    api_secret = "68taYQlneJG-JU2izvZsOY3L8sk"
)

## Selenium Webdriver

In [260]:
# ------------------------------------------------------------------------------
# 1. Set up Selenium WebDriver
# ------------------------------------------------------------------------------

chrome_options = Options() 
driver = webdriver.Chrome(options=chrome_options)

BASE_URL = "https://modulargrid.net"
SEARCH_URL = (
    "https://modulargrid.net/e/modules/browser?"
    "SearchName=&SearchVendor=&SearchFunction=&SearchSecondaryfunction=&SearchHeight=&SearchTe=&"
    "SearchTemethod=max&SearchBuildtype=a&SearchLifecycle=available&SearchSet=&SearchMarketplace=&"
    "SearchIsmodeled=0&SearchShowothers=0&SearchShowpanel=0&order=newest&direction=asc"
)

In [261]:
# ---------------------------------------
# 2. Load the initial search page
# ---------------------------------------

driver.get(SEARCH_URL)
time.sleep(3)

In [262]:
# ------------------------------------------------------------------------------
# 3. Scroll to load more modules
# ------------------------------------------------------------------------------

try:
    alphabetic_button = driver.find_element(By.CSS_SELECTOR, "a[data-search-order='alphabetic']")
    alphabetic_button.click()
    
except Exception as e:
    print("Alphabetic sort button not found or click failed:", e)

SCROLL_PAUSE_TIME = 1
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    try:
        wait_for_scroll(last_height, timeout=20)

    except Exception as e:
        print("No more new content loaded or timeout reached:", e)
        break

    last_height = driver.execute_script("return document.body.scrollHeight")

No more new content loaded or timeout reached: Message: 



In [264]:
# ---------------------------------------
# 4. Loop through each module to grab the link
# ---------------------------------------

module_data = []
box_modules = driver.find_elements(By.CLASS_NAME, "box-module")

for module in box_modules:
    try:
        module_id = module.get_attribute("data-module-id")
        link_element = module.find_element(By.TAG_NAME, "a")
        href = link_element.get_attribute("href")
        
        if href.startswith("https"):
            print(f"Found module ID {module_id} with link {href}")
            module_data.append({"id": module_id, "link": href})
    except Exception as e:
        print("No link found in this module:", e)

print("Total modules found:", len(module_data))
        

Found module ID 24577 with link https://modulargrid.net/e/paratek-%D0%A0%D0%98%D0%A2%D0%9C%D0%98%D0%9A%D0%A1-aluminium
Found module ID 24578 with link https://modulargrid.net/e/paratek-%D0%A0%D0%98%D0%A2%D0%9C%D0%98%D0%9A%D0%A1-black
Found module ID 26787 with link https://modulargrid.net/e/paratek-%D0%A0%D0%98%D0%A2%D0%9C%D0%98%D0%9A%D0%A1-black--
Found module ID 26788 with link https://modulargrid.net/e/paratek-%D0%A0%D0%98%D0%A2%D0%9C%D0%98%D0%9A%D0%A1-black-pink
Found module ID 29024 with link https://modulargrid.net/e/orpho-8-step
Found module ID 37431 with link https://modulargrid.net/e/synthrotek-adapt-1-4
Found module ID 12311 with link https://modulargrid.net/e/addac-system-addac812vu
Found module ID 51117 with link https://modulargrid.net/e/after-later-audio-ffs
Found module ID 26539 with link https://modulargrid.net/e/error-instruments-indian-resonator-v2
Found module ID 46869 with link https://modulargrid.net/e/apollo-view-modular-iou
Found module ID 10136 with link https:/

In [ ]:
# ------------------------------------------------------------------------------
# 5. Iterate through each module link to scrape vendor names
# ------------------------------------------------------------------------------

data = []

driver.set_page_load_timeout(10)
wait = WebDriverWait(driver, 10)

for module_info in module_data:
    url = module_info['link']
    id = module_info['id']

    try:
        driver.get(url)
    except TimeoutException:
        print(f"Skipping {url} due to timeout")
        continue

    print(url)

    try:
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".module-view-header h1")))
    except Exception as e:
        print(f"Timeout waiting for module name on {url}: {e}")

    #--------------------------------------- MODULE NAME ---------------------------------------
    try:
        module_name_element = driver.find_element(By.CSS_SELECTOR, ".module-view-header h1")
        module_name = module_name_element.text.strip()
    except Exception as e:
        print(f"Module name not found for {url}: {e}")
        module_name = ""

    # --------------------------------------- MANUFACTURER ---------------------------------------
    try:
        manufacturer_element = driver.find_element(By.CSS_SELECTOR, ".vendor-name")
        manufacturer = manufacturer_element.text.strip()
    except Exception as e:
        print(f"Manufacturer not found for {url}: {e}")
        manufacturer = ""

    # --------------------------------------- PRIMARY DESCRIPTION ---------------------------------------
    try:
        primary_description_element = driver.find_element(By.CSS_SELECTOR, "p.lead.wrap")
        primary_description = primary_description_element.text.strip()
    except Exception as e:
        print(f"Primary description not found for {url}: {e}")
        primary_description = ""

    # --------------------------------------- AVAILABILITY ---------------------------------------
    try:
        availability_elements = driver.find_elements(
            By.XPATH, 
            "//div[contains(@class, 'subspec')]/p[contains(@class, 'text-success') and contains(., 'currently available')]"
        )
        availability = 1 if availability_elements else 0
    except Exception as e:
        print(f"Error determining availability for {url}: {e}")
        availability = 0

    # --------------------------------------- APPROVED STAMP ---------------------------------------
    try:
        driver.find_element(By.CSS_SELECTOR, ".box-approved")
        approved_stamp = 1
    except Exception:
        approved_stamp = 0

    # --------------------------------------- PHYSICAL DIMENSIONS ---------------------------------------
    try:
        dims_dt = driver.find_element(By.XPATH, "//dt[normalize-space(text())='Dimensions']")
        parent_dl = dims_dt.find_element(By.XPATH, "./..")
        dd_elements = parent_dl.find_elements(By.TAG_NAME, "dd")
        dimensions_list = [dd.text.strip() for dd in dd_elements if dd.text.strip()]
        dimensions = " | ".join(dimensions_list)
    except Exception as e:
        print(f"Dimensions not found for {url}: {e}")
        dimensions = ""

    # --------------------------------------- POWER REQUIREMENTS ---------------------------------------
    try:
        current_draw_dt = driver.find_element(By.XPATH, "//dt[contains(., 'Current') and contains(., 'Draw')]")
        parent_dl = current_draw_dt.find_element(By.XPATH, "./..")
        dd_elements = parent_dl.find_elements(By.TAG_NAME, "dd")
        current_draw_list = [dd.text.strip() for dd in dd_elements if dd.text.strip()]
        power_requirements = " | ".join(current_draw_list)
    except Exception as e:
        print(f"Power requirement not found for {url}: {e}")
        power_requirements = ""

    # --------------------------------------- MODULE TAGS ---------------------------------------
    try:
        tags_div = driver.find_element(By.CSS_SELECTOR, "div.module-tags")
        tag_spans = tags_div.find_elements(By.CSS_SELECTOR, "span.label")
        module_tags = ", ".join([span.text.strip() for span in tag_spans if span.text.strip()])
    except Exception as e:
        print(f"Module tags not found for {url}: {e}")
        module_tags = ""

    # --------------------------------------- FULL DESCRIPTION ---------------------------------------
    try:
        module_details_div = driver.find_element(By.ID, "module-details")
        p_elements = module_details_div.find_elements(By.TAG_NAME, "p")
        full_desc_paragraphs = []
        
        for p in p_elements:
            p_classes = p.get_attribute("class") or ""
            if "lead" in p_classes and "wrap" in p_classes:
                continue
            
            if p.find_elements(By.TAG_NAME, "a"):
                continue
            
            text = p.text.strip()
            if text:
                full_desc_paragraphs.append(text)
        
        full_desc = "\n".join(full_desc_paragraphs)
    except Exception as e:
        print(f"Full description not found for {url}: {e}")
        full_desc = ""

    # --------------------------------------- PRICING INFORMATION (EURO) ---------------------------------------
    try:
        price_dd = driver.find_element(By.XPATH, "//dt[normalize-space(text())='Price']/following-sibling::dd")
        price_spans = price_dd.find_elements(By.XPATH, ".//span[contains(@class, 'currency-approx') or contains(@class, 'currency')]")
        price_texts = []
        for span in price_spans:
            text = span.text.strip()
            classes = span.get_attribute("class")
            if "currency-approx" in classes:
                text = "≈" + text
            price_texts.append(text)
        price_in_euro = " | ".join(price_texts)
    except Exception as e:
        print(f"Euro price not found for {url}: {e}")
        price_in_euro = ""

    # --------------------------------------- SWITCH TO USD AND SCRAPE PRICING ---------------------------------------
    try:
        # Click the USD link to switch currency
        usd_link = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@title, 'Display prices in $')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", usd_link)
        driver.execute_script("arguments[0].click();", usd_link)

        # Wait for the price to update to USD (optional)
        wait.until(lambda d: "$" in d.find_element(By.XPATH, "//dt[normalize-space(text())='Price']/following-sibling::dd").text)

        # Scrape USD pricing
        price_dd_usd = driver.find_element(By.XPATH, "//dt[normalize-space(text())='Price']/following-sibling::dd")
        price_spans_usd = price_dd_usd.find_elements(By.XPATH, ".//span[contains(@class, 'currency-approx') or contains(@class, 'currency')]")
        price_texts_usd = []
        for span in price_spans_usd:
            text = span.text.strip()
            classes = span.get_attribute("class")
            if "currency-approx" in classes:
                text = "≈" + text
            price_texts_usd.append(text)
        price_in_dollar = " | ".join(price_texts_usd)
    except Exception as e:
        print(f"Dollar price not found for {url}: {e}")
        price_in_dollar = ""

    # --------------------------------------- REVERT TO EURO CURRENCY ---------------------------------------
    try:
        eur_link = wait.until(
            EC.element_to_be_clickable((By.XPATH, "//a[contains(@title, 'Display prices in €')]"))
        )
        driver.execute_script("arguments[0].scrollIntoView();", eur_link)
        driver.execute_script("arguments[0].click();", eur_link)

        # Wait for the price to revert to Euro (optional)
        wait.until(lambda d: "€" in d.find_element(By.XPATH, "//dt[normalize-space(text())='Price']/following-sibling::dd").text)
    except Exception as e:
        print(f"Could not click to display Euro prices for {url}: {e}")

    # --------------------------------------- IMAGE EXTRACTION AND UPLOAD ---------------------------------------
    try:
        g_image_div = driver.find_element(By.CLASS_NAME, "g-image")
        a_tag = g_image_div.find_element(By.TAG_NAME, "a")
        relative_image_url = a_tag.get_attribute("href")

        if relative_image_url.startswith("/"):
            base_url = "https://modulargrid.net/"
            full_image_url = base_url + relative_image_url
        else:
            full_image_url = relative_image_url
        
        upload_result = cloudinary.uploader.upload(full_image_url)
        cloudinary_url = upload_result.get("secure_url", "")
    except Exception as e:
        print(f"Error uploading image for {url}: {e}")
        cloudinary_url = ""


    # --------------------------------------- STORE DATA ---------------------------------------
    data.append({
        "id": id,
        "module_name": module_name,
        "manufacturer": manufacturer,
        "primary_desc": primary_description,
        "available": availability,
        "approved_stamp": approved_stamp,
        "physical_dim": dimensions,
        "power_req": power_requirements,
        "module_tags": module_tags,
        "full_desc": full_desc,
        "price_in_euro": price_in_euro,
        "price_in_dollar": price_in_dollar,
        "module_url": url,
        "cloudinary_url": cloudinary_url
    })


In [ ]:
df = pd.DataFrame(data)
df

In [ ]:
print(df)

In [ ]:
df['available'].value_counts()

In [252]:
df.to_csv("sample.csv", index=False)